In [ ]:
!pip install -q accelerate -U
!pip install -q bitsandbytes -U
!pip install -q trl -U
!pip install -q peft -U
!pip install -q transformers -U
!pip install -q datasets -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import pandas as pd
from datasets import load_dataset, Dataset, dataset_dict

df = pd.read_csv('/content/amazon_product_details.csv', usecols=['category', 'about_product', 'product_name'])

In [ ]:
df['category']=df['category'].apply(lambda x: x.split('|')[-1])

In [ ]:
products = df[['category','product_name']]
description = df[['category','about_product']]

products=products.rename(columns={'product_name':'text'})
description=description.rename(columns={'about_product':'text'})

products['task_type']='product_name'
description['task_type']='product_description'

In [ ]:
df = pd.concat([products, description], ignore_index=True)

In [ ]:
dataset = Dataset.from_pandas(df)
dataset = dataset.shuffle(seed=0)
dataset = dataset.train_test_split(test_size=0.1)

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['category', 'text', 'task_type'],
        num_rows: 2637
    })
    test: Dataset({
        features: ['category', 'text', 'task_type'],
        num_rows: 293
    })
})

In [ ]:
dataset['test'][2]

{'category': 'InkjetPrinters',
 'text': 'Canon PIXMA MG2577s All-in-One Inkjet Colour Printer with 1 Additional Colour Cartridge',
 'task_type': 'product_name'}

In [ ]:
def formatting_func(example):
  text = f"""
                given the product category, you need to generate a '{example['task_type']}'.
                ###category:{example['category']}\n###{example['task_type']}: {example['text']}
          """
  return text

In [ ]:
print(formatting_func(dataset['train'][0]))


                given the product category, you need to generate a 'product_name'.
                ###category:SmartWatches
###product_name: PTron Newly Launched Force X10 Bluetooth Calling Smartwatch with 1.7" Full Touch Color Display, Real Heart Rate Monitor, SpO2, Watch Faces, 5 Days Runtime, Fitness Trackers & IP68 Waterproof (Blue)
          


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

base_model_id = "microsoft/phi-2"

model = AutoModelForCausalLM.from_pretrained(base_model_id, trust_remote_code=True, torch_dtype=torch.float16, load_in_8bit=True)
# tokenizer = AutoTokenizer.from_pretrained(base_model_id)

config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
tokenizer= AutoTokenizer.from_pretrained(
    base_model_id,
    padding_size='left',
    add_eos_token=True,
    add_bos_token=True,
    use_fast=False
    )

tokenizer.pad_token = tokenizer.eos_token

In [ ]:
max_length = 400

def tokenize(prompt):
  result = tokenizer(
      formatting_func(prompt),
      truncation=True,
      max_length=max_length,
      padding="max_length",
  )
  result['labels'] = result['input_ids'].copy()

  return result

In [ ]:
print(tokenize(dataset['train'][0]))

{'input_ids': [50256, 198, 50272, 35569, 262, 1720, 6536, 11, 345, 761, 284, 7716, 257, 705, 11167, 62, 3672, 4458, 198, 50272, 21017, 22872, 25, 25610, 54, 20981, 198, 21017, 11167, 62, 3672, 25, 19310, 1313, 43222, 19826, 1740, 5221, 1395, 940, 19263, 32677, 10880, 8340, 351, 352, 13, 22, 1, 6462, 15957, 5315, 16531, 11, 6416, 8894, 14806, 18289, 11, 1338, 46, 17, 11, 6305, 48463, 11, 642, 12579, 43160, 11, 34545, 17762, 364, 1222, 6101, 3104, 5638, 13288, 357, 14573, 8, 198, 50278, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,

In [ ]:
dataset = dataset.map(tokenize)

Map:   0%|          | 0/2637 [00:00<?, ? examples/s]

Map:   0%|          | 0/293 [00:00<?, ? examples/s]

In [ ]:
eval_prompt = """
                         Given the product category, you need to generate a 'Product Desciption'.
                         ### Category : BatteryChargers
                         ### Product Description:
                         """

In [ ]:
model_input = tokenizer(
    eval_prompt,
    truncation=True,
    max_length=max_length,
    padding="max_length",
    return_tensors="pt"
).to("cuda")


In [ ]:
model.eval()

with torch.no_grad():
  output = model.generate(**model_input, max_new_tokens=256, repetition_penalty=1.15)

print(tokenizer.decode(output[0], skip_special_tokens=True))

# Instead of printing 'result', print the decoded output directly.
# print(result)
print(tokenizer.decode(output[0])) # This will print the entire output, including special tokens

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



                           Given the product category, you need to generate a 'Product Desciption'.
                           ### Category : BatteryChargers
                           ### Product Description:
                           
      1. The battery charger is designed for use with all types of batteries including AA, AAA, C and D cell batteries. 

      2. It has an automatic shut-off feature that turns off after charging is complete.

      3. This device can be used both indoors and outdoors due to its weatherproof design.

      4. It comes with two USB ports which allows it to charge multiple devices at once.

      5. The charger also features a built-in LED indicator light that shows when the battery is fully charged.

      6. It's compact size makes it easy to store in any location.

      7. The charger includes a one year warranty against defects in materials or workmanship.

      8. It's compatible with most smartphones, tablets, laptops, and other electronic dev

In [ ]:
from peft import LoraConfig, get_peft_model

target_modules = ["wqkv", "fc1", "fc2"]

config = LoraConfig(
    r=32,
    lora_alpha=64,
    target_modules=target_modules,
    bias ="none",
    lora_dropout=0.05,
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, config)


In [ ]:
def print_trainable_parameters(model):
  """
  prints the number of trainable parameters in the model.

  """
  trainable_params = 0
  all_param = 0
  for _, param in model.named_parameters():
    all_param += param.numel()
    if param.requires_grad:
      trainable_params += param.numel()
  print(
      f"trainable params: {trainable_params} || all params:{all_param} || trainable%: {100 * trainable_params/all_param}"
  )

In [ ]:
print_trainable_parameters(model)

trainable params: 26214400 || all params:2805898240 || trainable%: 0.9342605382581515


In [ ]:
from accelerate import Accelerator

accelerator = Accelerator(gradient_accumulation_steps=1)
model = accelerator.prepare_model(model)

In [ ]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling
from datetime import datetime

In [ ]:
project = "phi2-finetune"
run_name = 'train-dir'
output_dir = "./" +run_name

In [35]:
args=TrainingArguments(
    output_dir = output_dir,
    per_device_train_batch_size = 1,

)

In [38]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [43]:
args=TrainingArguments(
    output_dir = output_dir,
    per_device_train_batch_size = 2,
    gradient_accumulation_steps=1,
    max_steps=100,
    learning_rate=2.5e-5, #want a small lr for ft
    optim="paged_adamw_8bit",
    logging_steps=25,
    logging_dir="./logs",
    save_strategy="steps",
    save_steps=25,
    evaluation_strategy="steps",
    eval_steps=25,
    do_eval=True,
    report_to="none"
)

trainer =Trainer(
    model=model,
    args=args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),

)

model.config.use_cache = False #silence the warning re-enable for inference

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss
25,2.739500,2.753135
50,2.696900,2.722595
75,2.598700,2.712447
100,2.586700,2.708579


TrainOutput(global_step=100, training_loss=2.6554427337646485, metrics={'train_runtime': 285.6837, 'train_samples_per_second': 0.7, 'train_steps_per_second': 0.35, 'total_flos': 1283916595200000.0, 'train_loss': 2.6554427337646485, 'epoch': 0.0758150113722517})

In [44]:
import torch
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    trust_remote_code=True,
    torch_dtype=torch.float16,
    load_in_8bit=True,

)

eval_tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    add_bos_token= True,
    trust_remote_code=True,
    padding_size='left',
    use_fast=False
)

eval_tokenizer.pad_token = eval_tokenizer.eos_token


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now default to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [45]:
from peft import PeftModel

ft_model = PeftModel.from_pretrained(base_model, '/content/train-dir/checkpoint-100')

In [56]:
eval_prompt = """
                         Given the product category, you need to generate a 'Product Description'.
                         ### Category : BatteryChargers
                         ### Product Description:
                         """
# Move model_input to the same device as the model (GPU)
model_input = eval_tokenizer(eval_prompt, return_tensors="pt")

ft_model.eval()
with torch.no_grad():
  output = ft_model.generate(**model_input, max_new_tokens=256, repetition_penalty=1.15)

  result = eval_tokenizer.decode(output[0], skip_special_tokens=True)

  print(result)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:2134: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [ ]:
!pip install sagemaker transformers datasets accelerate bitsandbytes trl peft -U

import sagemaker
import boto3
from sagemaker.huggingface import HuggingFace

# Define SageMaker session and role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# Define training parameters
hyperparameters = {
    "epochs": 1,  # Adjust as needed
    "per_device_train_batch_size": 4,  # Adjust as needed
    "gradient_accumulation_steps": 1,
    "max_steps": 1000,  # Adjust as needed
    "learning_rate": 1e-5,  # Adjust as needed
    "optim": "adamw_hf",  # Adjust as needed
    "logging_steps": 25,
    "save_strategy": "steps",
    "save_steps": 25,
    "evaluation_strategy": "steps",
    "eval_steps": 25,
    "do_eval": True,
    "report_to": "none",
    "model_id": "microsoft/phi-2", # Base model ID
    "fp16": True, # Enable mixed precision training
}

# Define training script (adapt from your existing notebook)
training_script = """
import pandas as pd
from datasets import load_dataset, Dataset, dataset_dict
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from peft import LoraConfig, get_peft_model, PeftModel
import os

def formatting_func(example):
    text = f"""
                given the product category, you need to generate a '{example['task_type']}'.
                ###category:{example['category']}\\n###{example['task_type']}: {example['text']}
          """
    return text

def tokenize(prompt):
    result = tokenizer(
        formatting_func(prompt),
        truncation=True,
        max_length=400,
        padding="max_length",
    )
    result['labels'] = result['input_ids'].copy()
    return result

def train():
    # Load dataset (replace with your dataset loading logic)
    df = pd.read_csv('/opt/ml/input/data/training/amazon_product_details.csv', usecols=['category', 'about_product', 'product_name'])
    df['category'] = df['category'].apply(lambda x: x.split('|')[-1])
    products = df[['category', 'product_name']]
    description = df[['category', 'about_product']]
    products = products.rename(columns={'product_name': 'text'})
    description = description.rename(columns={'about_product': 'text'})
    products['task_type'] = 'product_name'
    description['task_type'] = 'product_description'
    df = pd.concat([products, description], ignore_index=True)
    dataset = Dataset.from_pandas(df)
    dataset = dataset.shuffle(seed=0)
    dataset = dataset.train_test_split(test_size=0.1)

    # Load tokenizer and model
    base_model_id = os.environ.get("SM_CHANNEL_MODEL", "microsoft/phi-2")
    tokenizer = AutoTokenizer.from_pretrained(
        base_model_id,
        padding_size='left',
        add_eos_token=True,
        add_bos_token=True,
        use_fast=False
    )
    tokenizer.pad_token = tokenizer.eos_token

    model = AutoModelForCausalLM.from_pretrained(
        base_model_id,
        trust_remote_code=True,
        torch_dtype=torch.float16,
        load_in_8bit=True
    )

    # Apply LoRA
    target_modules = ["wqkv", "fc1", "fc2"]
    config = LoraConfig(
        r=32,
        lora_alpha=64,
        target_modules=target_modules,
        bias="none",
        lora_dropout=0.05,
        task_type="CAUSAL_LM"
    )
    model = get_peft_model(model, config)

    # Tokenize dataset
    dataset = dataset.map(tokenize)

    # Define training arguments
    training_args = TrainingArguments(
        output_dir="/opt/ml/model",  # Save model to SageMaker output directory
        **{k: v for k, v in hyperparameters.items() if k != "model_id"} # Pass hyperparameters
    )

    # Create Trainer and train
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset["train"],
        eval_dataset=dataset["test"],
        data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
    )

    model.config.use_cache = False
    trainer.train()

    # Save the PEFT model
    model.save_pretrained("/opt/ml/model")

if __name__ == "__main__":
    train()
"""

# Upload training script to S3
with open("train.py", "w") as f:
    f.write(training_script)

training_input_path = sagemaker_session.upload_data(
    path="/content/amazon_product_details.csv", # Replace with your dataset path
    key_prefix="training_data",
)

# Define HuggingFace estimator
huggingface_estimator = HuggingFace(
    entry_point="train.py",
    source_dir="./", # Directory containing the training script
    instance_type="ml.g4dn.xlarge", # Choose appropriate instance type
    instance_count=1,
    role=role,
    transformers_version="4.29",
    pytorch_version="1.13",
    py_version="py39",
    hyperparameters=hyperparameters,
)

# Start training job
huggingface_estimator.fit({"training": training_input_path})

# Deploy the model (optional)
predictor = huggingface_estimator.deploy(
    initial_instance_count=1, instance_type="ml.g4dn.xlarge"
)

In [58]:
!zip -r phi2_qlora_adapater.zip /content/train-dir/checkpoint-100

  adding: content/train-dir/checkpoint-100/ (stored 0%)
  adding: content/train-dir/checkpoint-100/adapter_config.json (deflated 53%)
  adding: content/train-dir/checkpoint-100/scheduler.pt (deflated 56%)
  adding: content/train-dir/checkpoint-100/trainer_state.json (deflated 69%)
  adding: content/train-dir/checkpoint-100/optimizer.pt (deflated 10%)
  adding: content/train-dir/checkpoint-100/README.md (deflated 66%)
  adding: content/train-dir/checkpoint-100/training_args.bin (deflated 52%)
  adding: content/train-dir/checkpoint-100/rng_state.pth (deflated 25%)
  adding: content/train-dir/checkpoint-100/adapter_model.safetensors (deflated 7%)
